In [2]:
from packages import WindModel
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset

class ClimateINRDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        df = pd.read_csv(csv_path)

        self.lons = torch.tensor(df['longitude'].values, dtype=torch.float32)
        self.lats = torch.tensor(df['latitude'].values, dtype=torch.float32)
        self.times = torch.tensor(df['time'].values, dtype=torch.float32)

        # min-max 정규화
        self.lons = 2 * (self.lons - self.lons.min()) / (self.lons.max() - self.lons.min())-1
        self.lats = 2  * (self.lats - self.lats.min()) / (self.lats.max() - self.lats.min()) - 1
        self.time = 2 * (self.times - self.times.min()) / (self.times.max() - self.times.min()) - 1

        self.coords = torch.stack([self.lons, self.lats, self.times], dim=-1)

        # 출력값
        self.values = torch.tensor(df[['u_wind', 'relative_humidity']].values, dtype=torch.float32)

    def __len__(self):
        return len(self.coords)
    
    def __getitem__(self, idx):
        return self.coords[idx], self.values[idx]

In [3]:
def gradient(y, x):
    grad = torch.autograd.grad(
        outputs=y,
        inputs=x,
        grad_outputs=torch.ones_like(y),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    return grad

def laplace(y, x):
    grad = gradient(y, x)
    lap = 0.0
    for i in range(grad.shape[-1]):
        lap += torch.autograd.grad(
            grad[..., i],
            x,
            grad_outputs=torch.ones_like(grad[..., i]),
            create_graph=True,
            retain_graph=True,
            only_inputs=True
        )[0][..., i]
    return lap

In [4]:
dataset = ClimateINRDataset('data/high_res_climate.csv')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

for coords, values in dataloader:
    print('좌표 (lat, lon, time): ', coords.shape)
    print("출력 (u_wimd, humidity):", values.shape)
    break

좌표 (lat, lon, time):  torch.Size([32, 3])
출력 (u_wimd, humidity): torch.Size([32, 2])


In [5]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from packages import WindModel  # 너가 정의한 INR 모델
# from utils import gradient, laplace  # 혹은 직접 정의한 함수

# 하이퍼파라미터
total_steps = 500
steps_til_summary = 100
lr = 1e-4

# 모델 초기화
model = WindModel(num_frequencies=10).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 데이터 준비
model_input, ground_truth = next(iter(dataloader))
model_input, ground_truth = model_input.cuda(), ground_truth.cuda()
print(model_input.shape)

# 학습 루프
for step in range(total_steps):
    lon_lat = model_input[:, :2]   # (batch_size, 2)
    time = model_input[:, 2:]      # (batch_size, 1)

    model_output, coords = model(lon_lat, time)
    loss = ((model_output - ground_truth) ** 2).mean()

    # 시각화: 일정 step마다 출력
    if step % steps_til_summary == 0:
        print(f'Step [{step:3d}/{total_steps:3d}], loss {loss.item():.6f}')
        img_grad = gradient(model_output, coords)
        img_laplacian = laplace(model_output, coords)
        num_points = model_input.shape[0]

        # 출력값은 단일 변수라고 가정 (e.g. u_wind)
        u_wind = model_output[:, 0]
        humidity = model_output[:, 1]

        img_size = int(num_points ** 0.5)

        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        axes[0].imshow(u_wind.view(img_size, img_size).cpu().detach().numpy(), cmap='viridis')
        axes[0].set_title('U Wind')

        axes[1].imshow(img_grad.norm(dim=-1).view(img_size, img_size).cpu().detach().numpy(), cmap='plasma')
        axes[1].set_title('Gradient Norm')

        axes[2].imshow(img_laplacian.view(img_size, img_size).cpu().detach().numpy(), cmap='inferno')
        axes[2].set_title('Laplacian')

        plt.tight_layout()
        plt.show()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


torch.Size([32, 3])
Step [  0/500], loss 1845.663574


RuntimeError: shape '[5, 5]' is invalid for input of size 32

: 